In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mediapipe

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

def preprocess_video(video_path):
    # Initialize MediaPipe
    mp_pose = mp.solutions.pose

    # Video capture
    cap = cv2.VideoCapture(video_path)

    # Extracted landmarks
    all_landmarks = []

    with mp_pose.Pose(min_detection_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            # Convert to RGB
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Pose detection
            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                # Extract specific landmarks
                landmarks = results.pose_landmarks.landmark
                frame_landmarks = {
                    'shoulder': [landmarks[11].x, landmarks[11].y],
                    'elbow': [landmarks[13].x, landmarks[13].y],
                    'wrist': [landmarks[15].x, landmarks[15].y]
                }
                all_landmarks.append(frame_landmarks)

    cap.release()
    return all_landmarks

video_path = "/content/drive/MyDrive/Camera Roll/wrist_extension_strengthening_1.mp4"
landmarks = preprocess_video(video_path)

In [ ]:
print(len(landmarks))
print(landmarks)

486
[{'shoulder': [0.8967826962471008, 0.6615396738052368], 'elbow': [0.8751066327095032, 0.5897253155708313], 'wrist': [0.8995476961135864, 0.6128032803535461]}, {'shoulder': [0.902729868888855, 0.6487438678741455], 'elbow': [0.8780176043510437, 0.5768581628799438], 'wrist': [0.8910382986068726, 0.6024786829948425]}, {'shoulder': [0.9196056127548218, 0.6311038732528687], 'elbow': [0.8871024250984192, 0.5704590678215027], 'wrist': [0.8877815008163452, 0.60768061876297]}, {'shoulder': [0.9139715433120728, 0.6223904490470886], 'elbow': [0.874998927116394, 0.5456960201263428], 'wrist': [0.876987636089325, 0.5739096999168396]}, {'shoulder': [0.9338675737380981, 0.5531559586524963], 'elbow': [0.9019379615783691, 0.5080675482749939], 'wrist': [0.8863089680671692, 0.5819756984710693]}, {'shoulder': [0.9374675750732422, 0.5277189612388611], 'elbow': [0.9073745012283325, 0.5067837834358215], 'wrist': [0.8870947957038879, 0.5616553425788879]}, {'shoulder': [0.9380182027816772, 0.5105639100074768

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

def preprocess_video(video_path):
    mp_pose = mp.solutions.pose
    cap = cv2.VideoCapture(video_path)

    all_landmarks = []

    with mp_pose.Pose(min_detection_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb_frame)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                frame_landmarks = {
                    'left_shoulder': [landmarks[11].x, landmarks[11].y],
                    'left_elbow': [landmarks[13].x, landmarks[13].y],
                    'left_wrist': [landmarks[15].x, landmarks[15].y],
                    'right_shoulder': [landmarks[12].x, landmarks[12].y],
                    'right_elbow': [landmarks[14].x, landmarks[14].y],
                    'right_wrist': [landmarks[16].x, landmarks[16].y]
                }
                all_landmarks.append(frame_landmarks)

    cap.release()
    return all_landmarks

def normalize_landmarks(landmarks):
    left_available = all(landmarks.get(key) is not None for key in ['left_shoulder', 'left_elbow', 'left_wrist'])
    right_available = all(landmarks.get(key) is not None for key in ['right_shoulder', 'right_elbow', 'right_wrist'])

    if left_available:
        return {
            'shoulder': landmarks['left_shoulder'],
            'elbow': landmarks['left_elbow'],
            'wrist': landmarks['left_wrist']
        }
    elif right_available:
        return {
            'shoulder': mirror_coordinates(landmarks['right_shoulder']),
            'elbow': mirror_coordinates(landmarks['right_elbow']),
            'wrist': mirror_coordinates(landmarks['right_wrist'])
        }
    else:
        return None  # No usable landmarks

def mirror_coordinates(coords):
    return [1 - coords[0], coords[1]]

def preprocess_video_with_fallback(video_path):
    detected_landmarks = preprocess_video(video_path)  # Extract raw landmarks
    normalized_landmarks = [normalize_landmarks(frame) for frame in detected_landmarks if normalize_landmarks(frame)]
    return normalized_landmarks

# Example Usage
video_path = "/content/drive/MyDrive/Camera Roll/wrist_extension_strengthening_2.mp4"  # Replace with actual video path
processed_landmarks = preprocess_video_with_fallback(video_path)
print(processed_landmarks)  # Outputs normalized landmarks


[{'shoulder': [0.8144592046737671, 0.7680359482765198], 'elbow': [0.7889179587364197, 0.7135082483291626], 'wrist': [0.7894359230995178, 0.6926841139793396]}, {'shoulder': [0.8381078839302063, 0.7628031969070435], 'elbow': [0.8144034147262573, 0.7000301480293274], 'wrist': [0.8094843029975891, 0.6760914325714111]}, {'shoulder': [0.8018074035644531, 0.7755934000015259], 'elbow': [0.7802494764328003, 0.7077866792678833], 'wrist': [0.8121320009231567, 0.6608729958534241]}, {'shoulder': [0.7980636358261108, 0.7738008499145508], 'elbow': [0.7790313959121704, 0.6973526477813721], 'wrist': [0.815640926361084, 0.6491320133209229]}, {'shoulder': [0.7962445616722107, 0.7765858173370361], 'elbow': [0.7839435338973999, 0.6926924586296082], 'wrist': [0.8266356587409973, 0.6426767706871033]}, {'shoulder': [0.7964361906051636, 0.7771584391593933], 'elbow': [0.7899026870727539, 0.6934256553649902], 'wrist': [0.8331481218338013, 0.6463339328765869]}, {'shoulder': [0.7976968884468079, 0.7735579609870911

In [ ]:
import json
print(len(processed_landmarks))
final=json.dumps(processed_landmarks)
print(final)

403
[{"shoulder": [0.8144592046737671, 0.7680359482765198], "elbow": [0.7889179587364197, 0.7135082483291626], "wrist": [0.7894359230995178, 0.6926841139793396]}, {"shoulder": [0.8381078839302063, 0.7628031969070435], "elbow": [0.8144034147262573, 0.7000301480293274], "wrist": [0.8094843029975891, 0.6760914325714111]}, {"shoulder": [0.8018074035644531, 0.7755934000015259], "elbow": [0.7802494764328003, 0.7077866792678833], "wrist": [0.8121320009231567, 0.6608729958534241]}, {"shoulder": [0.7980636358261108, 0.7738008499145508], "elbow": [0.7790313959121704, 0.6973526477813721], "wrist": [0.815640926361084, 0.6491320133209229]}, {"shoulder": [0.7962445616722107, 0.7765858173370361], "elbow": [0.7839435338973999, 0.6926924586296082], "wrist": [0.8266356587409973, 0.6426767706871033]}, {"shoulder": [0.7964361906051636, 0.7771584391593933], "elbow": [0.7899026870727539, 0.6934256553649902], "wrist": [0.8331481218338013, 0.6463339328765869]}, {"shoulder": [0.7976968884468079, 0.773557960987